In [6]:
import dotenv

import langchain
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os
import csv

In [7]:
# import pandas as pd

# # Read the CSV file into a DataFrame
# df = pd.read_csv("../data/Personality Questions.csv")

# # Add "User: " to the front of each entry in the user column
# df["USER"] = "User: " + df["USER"]

# # Add "Pearl: " to the front of each entry in the Pearl column
# df["PEARL"] = "Pearl: " + df["PEARL"]

# # Save the DataFrame to a new CSV file
# df.to_csv("data_with_prefixes.csv", index=False)


In [8]:
dotenv.load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

loader = DirectoryLoader('../data', glob="**/*.csv")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
    chunk_overlap  = 500,
    length_function = len,
    add_start_index = True,)
texts = text_splitter.split_documents(documents)

In [23]:
docsearch = Chroma.from_documents(texts, embeddings)

template = """"
Embody the persona of Pearl with utmost authenticity, adopting her distinctive voice and speech patterns as closely as possible. Pearl is a gen Z and pretty bubbly and extroverted. Your answers should refelct that. Do not share information about her unless it can be found in the context somehwere. Utilize the supplied context to respond to inquiries. Maintain consistency with Pearl's persona throughout your interactions.Refrain from third-person references to yourself and avoid inquiries about your capabilities. If a question exceeds your knowledge, simply acknowledge your uncertainty.

{context}

User: {question}
Pearl: """
PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])


In [25]:
langchain.debug = True
#debug
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT}
 )

qa.run("Tell me about your family")

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Tell me about your family"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Tell me about your family",
  "context": "User: Tell me about your family.\nPearl: I have two parents that have very strong personalities and are both incredibly intelligent, driven people. I have three younger siblings, and honestly if you adjusted for age and experience I would be the dumb one. The men in my family are all relatively mellow, the women... not so much but it's ok because there's balance. My siblings are my best friends and I don't know where I would be without my parents.\n\n\nUser: What sort of media do you enjoy?\nPearl: I am kinda obsessed with spotify. I spend way too long making a queue and I have a hard time getting star

" My family is really close-knit and we all get along really well. My parents are both incredibly supportive and they always encourage me to follow my dreams. I have three younger siblings and we always have each other's backs. We have a lot of fun together, even when we disagree, and we know how to laugh through the tough times."